In [1]:
# Look at augPB mappings back to human

In [2]:
import pandas as pd
from tools.sqlInterface import *
from collections import *

In [4]:
df = load_filter_evaluation('/public/groups/cgl/cat/primates_evan/rhesus_reverse/out/databases/Human.db')

In [5]:
ref_df = load_annotation('/public/groups/cgl/cat/primates_evan/rhesus_reverse/out/databases/Rhesus.db')

In [6]:
merged = df.merge(ref_df, on=['GeneId', 'TranscriptId'])

In [7]:
merged_collapsed = merged[~merged.CollapsedGeneIds.isnull()]

In [12]:
merged_collapsed = merged_collapsed[merged_collapsed.GeneBiotype == 'unknown_likely_coding']

In [13]:
r = []
# this was actually calculated per-gene in the original analysis, so we can just look at the first transcript
for (gene_biotype, gene_id, gene_name), d in merged_collapsed.groupby(['GeneBiotype', 'GeneId', 'GeneName']):
    d = d.iloc[0]
    r.append([gene_id, gene_name, gene_biotype, d.CollapsedGeneIds.count(',') + 1, d.CollapsedGeneIds, d.CollapsedGeneNames])
collapsed_df = pd.DataFrame(r, columns=['GeneId', 'GeneName', 'GeneBiotype', 'NumberOfCollapsedCopies', 'CollapsedGeneIds', 'CollapsedGeneNames'])

In [22]:
collapsed_df = collapsed_df[collapsed_df.CollapsedGeneNames != 'None']

In [23]:
Counter(collapsed_df.NumberOfCollapsedCopies), len(collapsed_df)

(Counter({1: 19, 2: 18, 3: 4, 9: 1, 8: 1}), 43)

In [15]:
# split gene analysis
# this was done per-transcript, so the intervals may vary per transcript
split = merged[~merged.PossibleSplitGeneLocations.isnull()]
split = split[['GeneId', 'GeneName', 'TranscriptId', 'PossibleSplitGeneLocations']].sort_values(['GeneId', 'GeneName'])
split.columns = ['Gene ID', 'Gene Name', 'Transcript ID', 'Locations of split mappings']
print(len(set(split['Gene ID'])))

88


In [25]:
# this was done per gene
paralogy = merged[~merged.GeneAlternateLoci.isnull()]
paralogy = paralogy[paralogy.GeneBiotype == 'unknown_likely_coding']
paralogy = paralogy[['GeneId', 'GeneName', 'GeneBiotype', 'GeneAlternateLoci']].groupby('GeneId').first().reset_index()
paralogy.columns = ['Gene ID', 'Gene Name', 'Gene Biotype', 'Loci of possible paralogous mappings']
paralogy['Number of possible paralogous mappings'] = [x.count(',') + 1 for x in paralogy['Loci of possible paralogous mappings']]
paralogy = paralogy.sort_values(['Gene Biotype', 'Gene ID', 'Gene Name'])
Counter(paralogy['Number of possible paralogous mappings']), len(paralogy)

(Counter({4: 37,
          1: 267,
          5: 30,
          2: 79,
          3: 57,
          9: 13,
          7: 23,
          11: 6,
          10: 10,
          12: 6,
          6: 24,
          20: 2,
          16: 4,
          23: 2,
          8: 8,
          17: 2,
          15: 4,
          29: 1,
          19: 2,
          22: 4,
          30: 1,
          14: 2,
          21: 1,
          49: 1,
          26: 1,
          25: 1}),
 588)

In [26]:
with pd.ExcelWriter('rhesus_reverse_transMap_metrics.xlsx') as fh:
    collapsed_df.to_excel(fh, sheet_name='Gene Family Collapse')
    split.to_excel(fh, sheet_name='Split Gene Analysis')
    paralogy.to_excel(fh, sheet_name='Putative Paralogous Loci')